In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the directory above the current notebook to the system path
sys.path.append(os.path.abspath(os.path.join('..')))

Folder Path

In [3]:
windows = True
folder_path = r"\\fs.ista.ac.at\drives\aventuri\archive\siegegrp\AlVe\MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"


Get the infoframe

In [4]:
from io_images import get_images_infoframe

In [ ]:
project_path = os.path.join(folder_path, "chunk_images")

infoframe = get_images_infoframe(project_path, 
                                 windows=windows, 
                                 conditions=['Age', 'Sex', 'Animal', 'Slide'], 
                                 extension="*.tif*")

In [ ]:
infoframe

Get the idx of folders

In [ ]:
# Get indices of folders containing '0030' to '0036'
mask_0030_0036 = infoframe['Folder'].str.contains(r'003[0-6]', regex=True)

# Get indices of folders containing '0037' to '0043'
mask_0037_0043 = infoframe['Folder'].str.contains(r'003[7-9]|004[0-3]', regex=True)

# Indices for each group
idx_0030_0036 = infoframe[mask_0030_0036].index.tolist()
idx_0037_0043 = infoframe[mask_0037_0043].index.tolist()

# Display the indices
print("Indices for folders containing '0030' to '0036':", idx_0030_0036)
print("Indices for folders containing '0037' to '0043':", idx_0037_0043)

min_1, max_1 = min(idx_0030_0036), max(idx_0030_0036)+1
min_2, max_2 = min(idx_0037_0043), max(idx_0037_0043)+1

print(min_1, max_1)
print(min_2, max_2)

Update the columns

In [ ]:
from utils import add_condition_columns
infoframe = add_condition_columns(infoframe,
                                age_values = [['18m', 0, max_2]],
                                sex_values = [['F', 0, max_2]],
                                animal_values = [['1', 0, max_1],
                                                ['2', min_2, max_2]])

In [ ]:
infoframe

Change name 

In [ ]:
from utils import update_file_name_and_path
project_path = folder_path = r"\\fs.ista.ac.at\drives\aventuri\archive\siegegrp\AlVe\MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"
infoframe = update_file_name_and_path(dataframe = infoframe, 
                                      folder_name = 'chunk_images', 
                                      project_path = project_path)

In [ ]:
infoframe

In [ ]:
infoframe['old_file_path'].iloc[0]

In [ ]:
infoframe['file_path'].iloc[0]

move chunk image

In [ ]:
from io_images import move_files

move_files(infoframe)

Correwct name 

In [ ]:
project_path = os.path.join(folder_path, "chunk_images")

infoframe = get_images_infoframe(project_path, 
                                 windows=windows, 
                                 conditions=['Age', 'Sex', 'Animal', 'Slide'], 
                                 extension="*.tif*")

In [ ]:
fn = infoframe.iloc[0]['file_path']

In [8]:
import re

def remove_duplicate(file_path):
    # The regex pattern will match a part like "microglia_Age_18m_Sex_F_Animal_1_Slide_0" or "microglia_Age_18m_Sex_F_Animal_2_Slide_8"
    pattern = r'(_Age_\d+m_Sex_[MF]_Animal_\d+_Slide_\d+)'
    
    # Use re.findall to find all occurrences of the repeating part
    matches = re.findall(pattern, file_path)
    
    if matches:
        # The first match will be the repeating part to remove (after its first occurrence)
        duplicate_part = matches[0]
        # Remove the second occurrence of this part in the filename
        return file_path.replace(duplicate_part, '', 1)
    
    return file_path